Last Name: Mukherjee
Date: 1/31/2019
Approach: A 2-layered perceptron or a 2-layered neural net. 
Estimated AUC: I expect the AUC on the test-set to be greater than 0.85. 

In [165]:
import pandas as pd
df_train=pd.read_csv("C:/Users/abhi0/Downloads/train.csv")
df_test=pd.read_csv("C:/Users/abhi0/Downloads/test.csv")

In [166]:
def DataPreProcess(df):
    
    #Getting dummies for 'device_type' feature
    df_dummies=pd.get_dummies(df['device_type'],prefix='category')
    df=pd.concat([df,df_dummies],axis=1)

    df=df.drop(['device_type'],axis=1)

    #filling the missing valeus in gender vector
    df=df.fillna('M')

    #label encoding gender
    from sklearn.preprocessing import LabelEncoder
    labelencoder_X = LabelEncoder()
    df['gender'] = labelencoder_X.fit_transform(df['gender'])

    #Normalizing the datasets
    #Same factor should be there for test and train
    df["income"] = df["income"] / df['income'].max()
    df["age"] = df["age"] / df["age"].max()
    df["cost_of_ad"] = df["cost_of_ad"] / df["cost_of_ad"].max()
    
    
    return df 

    

In [167]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2

In [168]:
#For training and testing the classifier
#df_processed_train -- pre-processed training data-set
#df_processed_test --- pre-processed test data-set

def buildingClassifier(df_processed_train,df_processed_test):
    
    ############### Separating the traning set into train and dev sets ##################
    
    #For the training data frame separating into dependent and independednt variables. 
    #Further,separating the dependednt variable into into training and dev.set
    #25-75 ratio adapted. 
    
    #Separating the independent variable:
    Y=df_processed_train['outcome']

    X=df_processed_train.drop(['outcome'],axis=1)
    
    #### Splitting the datasets
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_dev, y_train, y_dev = train_test_split(X, Y, test_size = 0.25, random_state = 0) 
    
    ################################# training the classifier ##################################
    
    #Parameters arrived after
    #grid search.
    roc_auc_test=[]
    Activation='sigmoid'
    Optimizer='rmsprop'
    BatchSize=50
    ActivityRegularizer=0.001
    
    # Initialising the ANN
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 13, kernel_initializer = 'uniform', activation = 'relu', input_dim = 13,
                         activity_regularizer=l2(ActivityRegularizer)))

    # Adding the second hidden layer
    classifier.add(Dense(units = 7,kernel_initializer = 'uniform', activation = 'relu',
                                    activity_regularizer=l2(ActivityRegularizer)))

    # Adding the output layer
    classifier.add(Dense(units = 1,kernel_initializer = 'uniform', activation ='sigmoid',
                                    activity_regularizer=l2(ActivityRegularizer)))

    # Compiling the ANN
    classifier.compile(optimizer = Optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

    # Fitting the ANN to the Training set
    classifier.fit(X_train, y_train, batch_size = BatchSize, epochs = 50)

    ##################### Making the predictions and evaluating the model ######################

    # Predicting the Test set results
    y_pred_dev = classifier.predict(X_dev)

    #AUC on the dev-set
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_dev, y_pred_dev)
    auc_devSet=auc(fpr, tpr)
    
    
    ##################################### Tesing the classifier #####3#############################
    
    # Predicting the Test set results
    X_test=df_processed_test
    
    #y_pred_test is the predicted outcome
    #from the pre-processed test set. 
    y_pred_test = classifier.predict(X_test)
    y_pred_test[y_pred_test>0.5]=1                
    y_pred_test[y_pred_test<=0.5]=0
    
    return y_pred_test

In [169]:
#Predicted Outcomes are the predicted results on the given test set i.e 'y_pred_test'
PredictedOutcomes=buildingClassifier(DataPreProcess(df_train),DataPreProcess(df_test))

Epoch 1/50
7500/7500 [==============================] - 1s 78us/step - loss: 0.6443 - accuracy: 0.9057
Epoch 2/50
7500/7500 [==============================] - 0s 34us/step - loss: 0.4913 - accuracy: 0.9057
Epoch 3/50
7500/7500 [==============================] - 0s 32us/step - loss: 0.4018 - accuracy: 0.9057
Epoch 4/50
7500/7500 [==============================] - 0s 33us/step - loss: 0.3564 - accuracy: 0.9057
Epoch 5/50
7500/7500 [==============================] - 0s 34us/step - loss: 0.3313 - accuracy: 0.9057
Epoch 6/50
7500/7500 [==============================] - 0s 34us/step - loss: 0.3158 - accuracy: 0.9057
Epoch 7/50
7500/7500 [==============================] - 0s 33us/step - loss: 0.3040 - accuracy: 0.9057
Epoch 8/50
7500/7500 [==============================] - 0s 33us/step - loss: 0.2959 - accuracy: 0.9057
Epoch 9/50
7500/7500 [==============================] - 0s 32us/step - loss: 0.2888 - accuracy: 0.9057
Epoch 10/50
7500/7500 [==============================] - 0s 34us/step - l

In [171]:
#Printing the first five predictions 

print(PredictedOutcomes[:5])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
